# Troubleshoot HDFS kerberos connections





## **Dependencies**

> This Notebook requires below installations.

----------------------------------------------------------------
<table>
<colgroup>
<col style="width: 10%" />
<col style="width: 10%" />
<col style="width: 10%" />
<col style="width: 10%" />
</colgroup>
<thead>
<tr class="header">
<th>Application</th>
<th>Required</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr class="odd">
<td><strong>java</strong></td>
<td>Yes</td>
<td>Java 7 or higher is required to run this tool. Please install it from (<a href="https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html">Java installation</a>)</td>
</tr>
<tr class="even">
<td><strong>java classpath setup</strong></td>
<td>Yes</td>
<td>If java is not setup in classpath please follow instruction <a href="https://docs.oracle.com/javase/tutorial/essential/environment/paths.html">Here</a> to setup.</td>
</tr>
</tbody>
</table>
<p>


Use this tool to test if the Polybase kerberos connection to your kerberos secure HDFS is successfull. This notebook requires about 40 MB of disk space. Learn more about  <a href="https://docs.microsoft.com/sql/relational-databases/polybase/polybase-troubleshoot-connectivity?view=sql-server-2017">HDFS and polybase kerberos troubleshooting.</a>
## <span style="color:red">Important Instructions</span>
### **Before you begin, you will need:**
* HDFS cluster name
* Kerberos username
* Kerberos password
* Kerberos realm
* kerberos KDC name or IP address

This independent tool provide complete details of secure connections which helps administrator to trouble shoot the kerberos connection issue.
(Note: This tool does not require Polybase feature and it does not depend on any SQL server components.)


## Function definition 
### (no user input required)

In [11]:

def PromptForInfo(promptMsg, errorMsg):
    
    promptResponse = input(promptMsg)
    if promptResponse == "":
        raise SystemExit(errorMsg + '\n')
    return promptResponse

def executeCommand(cmd, successMsgs, printMsg):
    print(printMsg)
    cmdOutput = !{cmd}
    cmdOutput = '\n'.join(cmdOutput)
    if any(msg in cmdOutput for msg in successMsgs):
        print(f"\nKerberos Success ")
    else:
        if any(msg in cmdOutput for msg in ['java.net.SocketTimeoutException: Receive timed out']):
            raise SystemExit(f'\nFailed due to wrong kdc hostname: Please check the kdc name and rerun the notebook after changing kdc hostname.\n' + cmd + '\n output:\n' + cmdOutput)
        if  any(msg in cmdOutput for msg in ['Client not found in Kerberos database (6) - CLIENT_NOT_FOUND']):
            raise SystemExit(f'\nThe admin Service Principal supplied does not exist in the realm. Please check if user exist in kerberos realm.\n' + cmd + '\n output:\n' + cmdOutput)
        if  any(msg in cmdOutput for msg in ['Checksum failed']):
            raise SystemExit(f'\n Bad password. Please check the password and re run the test with right password.\n' + cmd + '\n output:\n' + cmdOutput)
        if  any(msg in cmdOutput for msg in ['java.io.FileNotFoundException File does not exist']):
            raise SystemExit(f'\n specified file location does not exist on hdfs. change the location.\n' + cmd + '\n output:\n' + cmdOutput)
        if any(msg in cmdOutput for msg in ['not recongnized']):
            raise SystemExit(f'\n Java was not installed. Please check pre requisits and rerun this cell.\n' + cmd + '\n output:\n' + cmdOutput)
        if any(msg in cmdOutput for msg in ['java.net.ConnectException: Connection refused']):
            raise SystemExit(f'\n Port number is not correct. Please check hdfs connection port number is active.\n' + cmd + '\n output:\n' + cmdOutput)
        raise SystemExit(f'\nFailed during:\n\n\t{cmd}\n\nreturned: \n' + ''.join(cmdOutput) + '.\n')




## Check if Java installed.

In [12]:
cmd = f'java -version'
executeCommand(cmd, ['java version', 'build'], 'Checking whether java installed or not.')

Checking whether java installed or not.



Success >> java -version output:java version "1.8.0_221"
Java(TM) SE Runtime Environment (build 1.8.0_221-b11)
Java HotSpot(TM) 64-Bit Server VM (build 25.221-b11, mixed mode)


Enter hdfs hostname: (or IP address)


In [13]:
# Prompt user inputs:
hdfs_node_name = PromptForInfo('Please provide your cluster hdfs node IP address or host name ', 'Cluster Name node name is required!')

enter HDFS port number: Usually its 8020.

In [14]:
port_number = PromptForInfo('Please provide your Cluster hdfs listening port number', 'port number is required!')

Enter Kebreros realm name: upper case only (example: EXAMPLE.COM)

In [15]:
kerberos_realm = PromptForInfo('Please provide your Kerberos realm(UPPER CASE) ', 'Kerberos realm is required!')

Enter KDC server name/IP address (You can find this information in Ambari or hortonworks )

In [16]:
kerberos_kdc = PromptForInfo('Please provide your Kerberos KDC host name or IP ', 'kdc is required!')

Enter kerberos username: (Example: pdw_user)

In [21]:
kerberos_username = PromptForInfo('Please provide your Kerberos Username for login: ', 'Kerberos Username is required!')

Enter the file name which contains kerberos password. (You can edit the myKerbPass.txt in current working directory with password and provide myKerbPass.txt as input or enter complete path to the text file where password is stored.)

In [18]:
kerberos_password_file = PromptForInfo('Kerberos Password file name: ', 'Kerberos password file name is required!')

Run the test

In [22]:


cmd = f'java -classpath hdfs-kerberos-tester-0.0.1-jar-with-dependencies.jar com.microsoft.kerberos.client.HdfsBridge ' + hdfs_node_name + ' ' + port_number + ' ' + kerberos_realm + ' ' + kerberos_kdc+ ' '+ kerberos_username + ' ' + kerberos_password_file

executeCommand(cmd, ['CHECKPOINT 2 COMPLETED'], 'Testing Kebreros Connection')


Testing Kebreros Connection


SystemExit: 
The admin Service Principal supplied does not exist in the realm. Please check if user exisit in kerberos realm.
java -classpath hdfs-kerberos-tester-0.0.1-jar-with-dependencies.jar com.microsoft.kerberos.client.HdfsBridge 10.193.27.232 8020 APSHDPMS.COM 10.193.27.236 pdw_user1 myKerbPass.txt
 output:
Current working directory : f:\azure-data-studio\notebooks-python
0 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - Hostname: 10.193.27.232
2 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - Port: 8020
3 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - Kerberos Principal: pdw_user1
3 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - Kerberos Password File: myKerbPass.txt
3 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - CHECKPOINT 1 STARTING: Authentication against KDC server
3 [main] INFO com.microsoft.kerberos.client.HdfsBridge  - Filepath to test: /
805 [main] INFO org.apache.hadoop.conf.Configuration.deprecation  - fs.default.name is de

f:\azure-data-studio\notebooks-python\0.0.1\lib\site-packages\IPython\core\interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)





If you have installed Polybase feature, you can test Polybase configuration settings with this tool.

You can run below command and see if you can successfully connect to the kerberos HDFS server using polybase configuration.

1. Copy 'Hadoop' folder from Polybase installation folder to the current working directory of this notebook
3. Run below cell.


In [20]:
#cmd = f'java -classpath "hdfs-kerberos-tester-0.0.1.jar;.\Hadoop\HDP2_2\*;.\Hadoop\*;.\Hadoop\conf" com.microsoft.kerberos.client.HdfsBridge ' + hdfs_node_name + ' ' + port_number + ' ' + kerberos_realm + ' ' + kerberos_kdc+ ' '+ kerberos_username + ' ' + kerberos_password_file + ' ' 

# executeCommand(cmd, ['CHECKPOINT 2 COMPLETED'], 'Testing Kebreros Connection')